In [1]:
import numpy as np
import pandas as pd
import h5py
import pysam
import os
from modisco.visualization import viz_sequence
from modisco import util
from matplotlib import pyplot as plt
import pybedtools
from bs4 import BeautifulSoup

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [2]:
dttype = 'DNASE'
mode = "counts"
celltype="IMR90_new"
modisco_path = '/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/'+dttype+'/'+celltype+'/merge_folds_new_may_05_24/'+mode+'/modisco_old_format.h5'
ppm_dir = ''

htmld="/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/"+dttype+'/'+celltype+"/merge_folds_new_may_05_24/"+mode+"/motifs.html"
tomtom = pd.read_html(htmld)


background=[0.25, 0.25, 0.25, 0.25]

In [3]:
def trim_motif_new(cwm, motif, trim_threshold=0.20):
    """
    Given the PFM and motif (both L x 4 arrays) (the motif could be the
    PFM itself), trims `motif` by cutting off flanks of low information
    content in `pfm`. `min_ic` is the minimum required information
    content. If specified this trimmed motif will be extended on either
    side by `pad` bases.
    If no base passes the `min_ic` threshold, then no trimming is done.
    """
    
    score = np.sum(np.abs(cwm), axis=1)
    trim_thresh = np.max(score) * trim_threshold  # Cut off anything less than 30% of max score
    pass_inds = np.where(score >= trim_thresh)[0]
    trimmed = motif[np.min(pass_inds): np.max(pass_inds) + 1]
 
    if not trimmed.size:
        return motif
    
    return trimmed

def import_tfmodisco_motifs(tfm_results_path, trim=True, only_pos=True):
    """
    Imports the PFMs to into a dictionary, mapping `(x, y)` to the PFM,
    where `x` is the metacluster index and `y` is the pattern index.
    Arguments:
        `tfm_results_path`: path to HDF5 containing TF-MoDISco results
        `out_dir`: where to save motifs
        `trim`: if True, trim the motif flanks based on information content
        `only_pos`: if True, only return motifs with positive contributions
    Returns the dictionary of PFMs.
    """ 
    pfms = {}
    with h5py.File(tfm_results_path, "r") as f:
        metaclusters = f["metacluster_idx_to_submetacluster_results"]
        num_metaclusters = len(metaclusters.keys())
        for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
            metacluster = metaclusters[metacluster_key]
            if "patterns" not in metacluster["seqlets_to_patterns_result"]:
                continue
            patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
            num_patterns = len(patterns["all_pattern_names"][:])
            for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
#                pattern_name = pattern_name.decode()
                pattern_name = pattern_name

                pattern = patterns[pattern_name]
                pfm = pattern["sequence"]["fwd"][:]
                cwm = pattern["task0_contrib_scores"]["fwd"][:]
                
                # Check that the contribution scores are overall positive
                if only_pos and np.sum(cwm) < 0:
                    continue
                    
                if trim:
                    pfm = trim_motif_new(cwm, cwm)
                else:
                    pfm = cwm
                    
                pfms["%d_%d" % (metacluster_i,pattern_i)] = pfm
    return pfms

In [4]:
pfms = import_tfmodisco_motifs(modisco_path, trim=False)

OSError: Unable to open file (unable to open file: name = '/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/DNASE/IMR90_new/merge_folds_new_may_05_24/counts/modisco_old_format.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
for key in pfms:
    f = open(os.path.join(ppm_dir,mode+"_"+key+".pfm"),"w")
    #print(pfms[key])
    np.savetxt(f, pfms[key], fmt='%f')
    f.close()
    

In [ ]:
tomtom[0]

In [7]:

tomtom[0]["pattern"] = tomtom[0]["pattern"].str.replace("pos_patterns.pattern","0").str.replace("neg_patterns.pattern","1")


/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [8]:
tomtom[0]

,pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
0,0_0,43236,NaN,NaN,CTCF_MA0139.1,1.613360e-18,NaN,CTCF_HUMAN.H11MO.0.A,3.443150e-13,NaN,CTCF_MOUSE.H11MO.0.A,1.686330e-12,NaN
1,0_1,16197,NaN,NaN,PO5F1_HUMAN.H11MO.0.A,6.697680e-09,NaN,NANOG_HUMAN.H11MO.0.A,9.111410e-09,NaN,PO5F1_MOUSE.H11MO.0.A,1.914660e-08,NaN
2,0_2,15185,NaN,NaN,SP1_HUMAN.H11MO.0.A,3.897330e-07,NaN,SP3_HUMAN.H11MO.0.B,4.759600e-07,NaN,SP3_MOUSE.H11MO.0.B,4.759600e-07,NaN
3,0_3,13730,NaN,NaN,ATF3_MOUSE.H11MO.0.A,6.692760e-03,NaN,JUN_MOUSE.H11MO.0.A,6.692760e-03,NaN,JUNB_HUMAN.H11MO.0.A,6.692760e-03,NaN
4,0_4,11703,NaN,NaN,TEAD1_HUMAN.H11MO.0.A,2.788660e-03,NaN,TEAD4_HUMAN.H11MO.0.A,2.788660e-03,NaN,TEAD4_MOUSE.H11MO.0.A,2.788660e-03,NaN
5,0_5,8394,NaN,NaN,NANOG_MOUSE.H11MO.0.A,4.600070e-12,NaN,SOX2_MOUSE.H11MO.0.A,5.518060e-12,NaN,PO5F1_MOUSE.H11MO.0.A,1.067370e-08,NaN
6,0_6,5439,NaN,NaN,NFYB_HUMAN.H11MO.0.A,3.875130e-04,NaN,NFYB_MOUSE.H11MO.0.A,3.875130e-04,NaN,NFYC_HUMAN.H11MO.0.A,3.875130e-04,NaN
7,0_7,4339,NaN,NaN,ZIC3_HUMAN.H11MO.0.B,7.648100e-09,NaN,ZIC3_MOUSE.H11MO.0.A,7.648100e-09,NaN,ZIC2_MOUSE.H11MO.0.C,1.566590e-06,NaN
8,0_8,3167,NaN,NaN,ELK1_ETS_1,1.524760e-06,NaN,ELK3_ETS_1,1.524760e-06,NaN,ELK3_MA0759.1,1.524760e-06,NaN
9,0_9,2747,NaN,NaN,NRF1_MA0506.1,1.063850e-05,NaN,NRF1_MOUSE.H11MO.0.A,5.360800e-05,NaN,NRF1_NRF_1,5.360800e-05,NaN


In [9]:
tomtom[0][["pattern","num_seqlets"]].to_csv(os.path.join(ppm_dir,mode+"_counts.csv"),sep=",",index=False, header=False)